In [4]:
from jira import JIRA
import os

def create_jira_ticket(summary: str, description: str):
    options = {"server": os.getenv("JIRA_URL")}
    jira = JIRA(
        options,
        basic_auth=(os.getenv("JIRA_EMAIL"), os.getenv("JIRA_API_TOKEN"))
    )
    issue_dict = {
        'project': {'key': os.getenv("JIRA_PROJECT_KEY")},
        'summary': summary,
        'description': description,
        'issuetype': {'name': 'Task'},
    }
    issue = jira.create_issue(fields=issue_dict)
    print(f"Created Jira issue: {issue.key}")
    return issue.key


In [2]:

from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain_community.vectorstores import Qdrant
from langchain.text_splitter import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.runnables import RunnableLambda


load_dotenv()

def load_txt_as_documents(txt_file):
    with open(txt_file, 'r', encoding='utf-8') as f:
        raw_text = f.read()
    return raw_text


raw_text = load_txt_as_documents(r"C:\Users\FINE LAPTOP\Desktop\Langchain\langgraph\rag_service.txt")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_text(raw_text)
documents = [Document(page_content=chunk) for chunk in texts]


embedding_function = OpenAIEmbeddings()


qdrant_client = QdrantClient(host="localhost", port=6333)

qdrant_client.recreate_collection(
    collection_name="rag_txt_collection",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)


db = Qdrant(
    client=qdrant_client,
    collection_name="rag_txt_collection",
    embeddings=embedding_function
)


db.add_documents(documents)

print("Uploaded documents to running Docker Qdrant.")


from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI


class GraphState(TypedDict):
    question: str
    context: str
    answer: str


llm = ChatOpenAI(model="gpt-3.5-turbo")


def retrieve(state: GraphState):
    query = state["question"]
    retriever = db.as_retriever()
    docs = retriever.invoke(query)
    context = "\n\n".join([doc.page_content for doc in docs])
    return {"question": query, "context": context}




Uploaded documents to running Docker Qdrant.


In [5]:
def generate(state: GraphState):
    prompt = f"""Answer the question using this context:\n\n{state['context']}\n\nQuestion: {state['question']}"""
    response = llm.invoke(prompt)
    answer = response.content

    # Auto-create Jira ticket if keywords match
    trigger_keywords = ["issue", "problem", "bug", "error", "fail", "help", "support"]
    if any(word in state["question"].lower() for word in trigger_keywords):
        create_jira_ticket(
            summary=f"User Support Request: {state['question']}",
            description=f"""Auto-created from RAG system.

Question:
{state['question']}

Answer:
{answer}
"""
        )

    return {
        "question": state["question"],
        "context": state["context"],
        "answer": answer
    }


graph = StateGraph(GraphState)
graph.add_node("retrieve", RunnableLambda(retrieve))
graph.add_node("generate", RunnableLambda(generate))
graph.set_entry_point("retrieve")
graph.add_edge("retrieve", "generate")
graph.add_edge("generate", END)
app = graph.compile()


inputs = {"question": "I have an issue setting a different delivery address up"}
result = app.invoke(inputs)

print(result['answer'])

Created Jira issue: SCRUM-2
Answer: I understand that you're facing difficulties in setting up a different delivery address. I'm here to assist you in resolving this matter. To set up a new delivery address, please follow these steps:

1. Log in to your account.
2. Go to the 'Delivery Address' or 'Shipping Information' section.
3. Look for the option to add a new address or edit existing addresses.
4. Enter the details of the different delivery address you wish to set up.
5. Verify that all the information is correct.
6. Save the changes.

If you encounter any issues or need further guidance while setting up the new delivery address, please don't hesitate to reach out to me. I'm here to help you every step of the way.
